# Performance Analysis of Professional Cycling Riders
## A Statistical Investigation of Rider Classes and Stage Performance

**Author:** Ali Mohamed Elasfar  
**Email:** ali.elasfar9@gmail.com  
**Date:** January 2026  
**Institution:** TU Dortmund University - M.Sc. Data Science Application

---

### Research Questions
1. **RQ1:** Is there a statistically significant difference in performance between the four rider classes?
2. **RQ2:** Does performance differ across stage classes for each rider class?

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tests
from scipy import stats
from scipy.stats import shapiro, kruskal, mannwhitneyu
from itertools import combinations

# Settings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load the dataset (space-separated)
# Update this path to your file location
df = pd.read_csv('data_report.csv', sep=' ')

print("Dataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} observations × {df.shape[1]} variables")

In [ ]:
# First 10 rows
df.head(10)

In [ ]:
# Data types
df.dtypes

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())
print(f"\nTotal missing: {df.isnull().sum().sum()}")

In [ ]:
# Unique values per variable
print("Unique Values:")
for col in df.columns:
    print(f"  {col}: {df[col].nunique()}")

## 3. Data Summary

In [ ]:
# Define ordering for consistent results
rider_order = ['All Rounder', 'Climber', 'Sprinter', 'Unclassed']
stage_order = ['flat', 'hills', 'mount']
stage_labels = {'flat': 'Flat', 'hills': 'Hills', 'mount': 'Mountain'}

In [ ]:
# Rider class distribution (unique riders)
print("Rider Class Distribution (unique riders):")
rider_counts = df.groupby('rider_class')['all_riders'].nunique().reindex(rider_order)
print(rider_counts)
print(f"\nTotal riders: {rider_counts.sum()}")

In [ ]:
# Stage class distribution (observations)
print("Stage Class Distribution (observations):")
print(df['stage_class'].value_counts())

## 4. Descriptive Statistics

### 4.1 Overall Distribution of Points

In [ ]:
# Overall summary statistics
overall_stats = pd.DataFrame({
    'Statistic': ['n', 'Mean', 'Median', 'Std. Dev.', 'Variance', 
                  'Minimum', 'Maximum', 'Skewness', 'Kurtosis'],
    'Value': [
        len(df),
        df['points'].mean(),
        df['points'].median(),
        df['points'].std(),
        df['points'].var(),
        df['points'].min(),
        df['points'].max(),
        df['points'].skew(),
        df['points'].kurtosis()
    ]
})

print("Table 2: Summary statistics for points (n = 3,496)")
print("="*50)
overall_stats

In [ ]:
# Percentage of zero-point observations
zero_pct = (df['points'] == 0).sum() / len(df) * 100
print(f"Percentage of zero-point observations: {zero_pct:.1f}%")

### 4.2 Performance by Rider Class

In [ ]:
# Descriptive statistics by rider class
rider_stats = df.groupby('rider_class')['points'].agg([
    ('n', 'count'),
    ('Mean', 'mean'),
    ('SD', 'std'),
    ('Median', 'median'),
    ('IQR', lambda x: x.quantile(0.75) - x.quantile(0.25)),
    ('Max', 'max')
]).reindex(rider_order)

print("Table 3: Descriptive statistics of points by rider class")
print("="*70)
rider_stats.round(2)

In [ ]:
# Zero-point percentages by rider class
print("Zero-Point Percentages by Rider Class:")
print("-"*40)
for rc in rider_order:
    rc_data = df[df['rider_class'] == rc]['points']
    zero_pct = (rc_data == 0).sum() / len(rc_data) * 100
    print(f"  {rc}: {zero_pct:.1f}%")

### 4.3 Interaction Between Rider Class and Stage Class

In [ ]:
# Mean points by rider class and stage class
interaction_means = df.pivot_table(
    values='points', 
    index='stage_class', 
    columns='rider_class', 
    aggfunc='mean'
).reindex(index=stage_order, columns=rider_order)

# Display with proper labels
interaction_display = interaction_means.copy()
interaction_display.index = [stage_labels[s] for s in interaction_display.index]

print("Table 4: Mean points by rider class and stage class")
print("="*60)
interaction_display.round(2)

## 5. Visualizations

### Figure 1: Pie Chart - Rider Class Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(9, 7))

colors_pie = ['#e74c3c', '#2ecc71', '#3498db', '#9b59b6']
explode = (0.05, 0.02, 0.02, 0)

wedges, texts, autotexts = ax.pie(
    rider_counts, 
    labels=[f'{rc}\n({count} riders)' for rc, count in zip(rider_order, rider_counts)],
    autopct='%1.1f%%',
    colors=colors_pie, 
    explode=explode, 
    startangle=90,
    textprops={'fontsize': 11}
)

for autotext in autotexts:
    autotext.set_fontweight('bold')
    autotext.set_fontsize(12)

ax.set_title('Distribution of Riders by Class (n=184)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('fig1_pie_chart.pdf', dpi=300, bbox_inches='tight')
plt.show()

### Figure 2: Violin Plot - Distribution by Rider Class

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors_violin = {'All Rounder': '#e74c3c', 'Climber': '#2ecc71', 
                 'Sprinter': '#3498db', 'Unclassed': '#9b59b6'}

violin_data = [df[df['rider_class'] == rc]['points'].values for rc in rider_order]

parts = ax.violinplot(violin_data, positions=range(len(rider_order)), 
                      showmeans=True, showmedians=True)

# Color the violins
for i, pc in enumerate(parts['bodies']):
    pc.set_facecolor(list(colors_violin.values())[i])
    pc.set_alpha(0.7)

parts['cmeans'].set_color('red')
parts['cmeans'].set_linewidth(2)
parts['cmedians'].set_color('black')
parts['cmedians'].set_linewidth(2)

ax.set_xticks(range(len(rider_order)))
ax.set_xticklabels(rider_order, fontsize=11)
ax.set_xlabel('Rider Class', fontsize=12, fontweight='bold')
ax.set_ylabel('Points', fontsize=12, fontweight='bold')
ax.set_title('Distribution of Points by Rider Class (Violin Plot)', fontsize=14, fontweight='bold')
ax.set_ylim(-10, 320)

# Legend
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], color='red', linewidth=2, label='Mean'),
                   Line2D([0], [0], color='black', linewidth=2, label='Median')]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig('fig2_violin_plot.pdf', dpi=300, bbox_inches='tight')
plt.show()

### Figure 3: Heatmap - Mean Points Interaction

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

heatmap_data = interaction_means.copy()
heatmap_data.index = ['Flat', 'Hills', 'Mountain']

sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='YlOrRd', 
            linewidths=0.5, ax=ax, cbar_kws={'label': 'Mean Points'},
            annot_kws={'size': 12, 'weight': 'bold'})

ax.set_xlabel('Rider Class', fontsize=12, fontweight='bold')
ax.set_ylabel('Stage Class', fontsize=12, fontweight='bold')
ax.set_title('Heatmap of Mean Points by Rider and Stage Class', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('fig3_heatmap.pdf', dpi=300, bbox_inches='tight')
plt.show()

### Figure 4: Bar Chart - Mean Points by Groups

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(rider_order))
width = 0.25

means_flat = [interaction_means.loc['flat', rc] for rc in rider_order]
means_hills = [interaction_means.loc['hills', rc] for rc in rider_order]
means_mount = [interaction_means.loc['mount', rc] for rc in rider_order]

bars1 = ax.bar(x - width, means_flat, width, label='Flat', color='#3498db', edgecolor='black', linewidth=0.5)
bars2 = ax.bar(x, means_hills, width, label='Hills', color='#2ecc71', edgecolor='black', linewidth=0.5)
bars3 = ax.bar(x + width, means_mount, width, label='Mountain', color='#e74c3c', edgecolor='black', linewidth=0.5)

ax.set_xlabel('Rider Class', fontsize=12, fontweight='bold')
ax.set_ylabel('Mean Points', fontsize=12, fontweight='bold')
ax.set_title('Mean Points by Rider Class and Stage Class', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(rider_order, fontsize=11)
ax.legend(title='Stage Class', fontsize=10)
ax.set_ylim(0, 75)

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('fig4_barchart.pdf', dpi=300, bbox_inches='tight')
plt.show()

### Figure 5: Box Plot - Grouped Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

sns.boxplot(data=df, x='rider_class', y='points', hue='stage_class', 
            order=rider_order, hue_order=stage_order,
            palette=['#3498db', '#2ecc71', '#e74c3c'], ax=ax)

ax.set_xlabel('Rider Class', fontsize=12, fontweight='bold')
ax.set_ylabel('Points', fontsize=12, fontweight='bold')
ax.set_title('Distribution of Points by Rider Class and Stage Class', fontsize=14, fontweight='bold')
ax.legend(title='Stage Class', labels=['Flat', 'Hills', 'Mountain'])

plt.tight_layout()
plt.savefig('fig5_boxplot.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 6. Hypothesis Testing

### 6.1 Assessment of Normality (Shapiro-Wilk Test)

**Hypotheses:**
- H₀: The data are normally distributed
- H₁: The data are not normally distributed

**Significance level:** α = 0.05

In [ ]:
print("Shapiro-Wilk Test Results")
print("="*60)
print(f"{'Rider Class':<15} {'W statistic':<15} {'p-value':<15} {'Decision'}")
print("-"*60)

shapiro_results = {}

for rc in rider_order:
    data = df[df['rider_class'] == rc]['points'].values
    
    # Shapiro-Wilk (sample if too large)
    if len(data) > 5000:
        sample = np.random.choice(data, 5000, replace=False)
    else:
        sample = data
    
    stat, p_value = shapiro(sample)
    decision = "Reject H₀" if p_value < 0.05 else "Fail to reject H₀"
    shapiro_results[rc] = {'W': stat, 'p': p_value}
    
    p_str = "< 0.001" if p_value < 0.001 else f"{p_value:.4f}"
    print(f"{rc:<15} {stat:<15.3f} {p_str:<15} {decision}")

print("-"*60)
print("\nConclusion: All groups significantly deviate from normality (p < 0.001).")
print("Non-parametric tests will be employed.")

### 6.2 RQ1: Differences Between Rider Classes (Kruskal-Wallis Test)

**Hypotheses:**
- H₀: The distributions of all groups are identical
- H₁: At least one group has a different distribution

**Test:** Kruskal-Wallis H Test (non-parametric alternative to one-way ANOVA)

In [ ]:
# Prepare groups
groups = [df[df['rider_class'] == rc]['points'].values for rc in rider_order]

# Kruskal-Wallis test
H_stat, p_value_kw = kruskal(*groups)

# Effect size (eta-squared)
k = len(rider_order)  # number of groups
N = len(df)           # total sample size
eta_squared = (H_stat - k + 1) / (N - k)

print("Kruskal-Wallis H Test Results")
print("="*50)
print(f"H statistic: {H_stat:.2f}")
print(f"Degrees of freedom: {k-1}")
print(f"p-value: < 0.001")
print(f"\nEffect size (η²_H): {eta_squared:.3f}")

# Interpret effect size
if eta_squared < 0.01:
    effect_interp = "small"
elif eta_squared < 0.06:
    effect_interp = "small-to-medium"
elif eta_squared < 0.14:
    effect_interp = "medium-to-large"
else:
    effect_interp = "large"

print(f"Effect size interpretation: {effect_interp} (Cohen, 1988)")
print("\n" + "="*50)
print("CONCLUSION: Reject H₀. Significant differences exist between rider classes.")

### 6.3 Post-hoc Pairwise Comparisons (Mann-Whitney U with Bonferroni)

In [ ]:
# Number of comparisons
m = len(list(combinations(rider_order, 2)))
alpha_adj = 0.05 / m

print("Post-hoc Pairwise Comparisons")
print("="*70)
print(f"Number of comparisons: {m}")
print(f"Bonferroni-adjusted α: {alpha_adj:.4f}")
print()

print("Table 5: Pairwise Mann-Whitney U tests with Bonferroni correction")
print("-"*70)
print(f"{'Comparison':<30} {'U statistic':<15} {'Adj. p-value':<15} {'Sig.'}")
print("-"*70)

posthoc_results = []

for rc1, rc2 in combinations(rider_order, 2):
    data1 = df[df['rider_class'] == rc1]['points'].values
    data2 = df[df['rider_class'] == rc2]['points'].values
    
    U_stat, p_value = mannwhitneyu(data1, data2, alternative='two-sided')
    p_adj = min(p_value * m, 1.0)  # Bonferroni adjustment
    
    # Significance stars
    if p_adj < 0.001:
        sig = "***"
    elif p_adj < 0.01:
        sig = "**"
    elif p_adj < 0.05:
        sig = "*"
    else:
        sig = "ns"
    
    posthoc_results.append({
        'Comparison': f"{rc1} vs {rc2}",
        'U': U_stat,
        'p_adj': p_adj,
        'sig': sig
    })
    
    p_str = "< 0.001" if p_adj < 0.001 else f"{p_adj:.4f}"
    print(f"{rc1} vs {rc2:<15} {U_stat:<15,.0f} {p_str:<15} {sig}")

print("-"*70)
print("Note: * p < 0.05; ** p < 0.01; *** p < 0.001")
print("\nCONCLUSION: All pairwise comparisons are statistically significant.")
print("Performance hierarchy: All Rounder > Climber > Sprinter > Unclassed")

### 6.4 RQ2: Performance Across Stage Classes

#### 6.4.1 Rider class differences within each stage class

In [ ]:
print("Table 6: Kruskal-Wallis tests - Rider classes within each stage class")
print("="*70)
print(f"{'Stage Class':<15} {'H':<15} {'df':<10} {'p-value':<15} {'Result'}")
print("-"*70)

for sc in stage_order:
    sc_data = df[df['stage_class'] == sc]
    groups_sc = [sc_data[sc_data['rider_class'] == rc]['points'].values for rc in rider_order]
    
    H_sc, p_sc = kruskal(*groups_sc)
    
    p_str = "< 0.001" if p_sc < 0.001 else f"{p_sc:.4f}"
    result = "Significant" if p_sc < 0.05 else "Not significant"
    print(f"{stage_labels[sc]:<15} {H_sc:<15.2f} {k-1:<10} {p_str:<15} {result}")

print("-"*70)

#### 6.4.2 Stage class effects within each rider class

In [ ]:
print("Table 7: Kruskal-Wallis tests - Stage classes within each rider class")
print("="*70)
print(f"{'Rider Class':<15} {'H':<15} {'df':<10} {'p-value':<15} {'Result'}")
print("-"*70)

rider_kw_results = {}

for rc in rider_order:
    rc_data = df[df['rider_class'] == rc]
    groups_rc = [rc_data[rc_data['stage_class'] == sc]['points'].values for sc in stage_order]
    
    H_rc, p_rc = kruskal(*groups_rc)
    rider_kw_results[rc] = {'H': H_rc, 'p': p_rc}
    
    p_str = "< 0.001" if p_rc < 0.001 else f"{p_rc:.4f}"
    result = "Significant" if p_rc < 0.05 else "Not significant"
    print(f"{rc:<15} {H_rc:<15.2f} {2:<10} {p_str:<15} {result}")

print("-"*70)
print(f"\nSprinters show the strongest stage-dependency (H = {rider_kw_results['Sprinter']['H']:.2f})")

## 7. Summary of Results

In [ ]:
print("="*80)
print("SUMMARY OF RESULTS")
print("="*80)

print("\n--- RQ1: Differences Between Rider Classes ---")
print(f"• Kruskal-Wallis test: H = {H_stat:.2f}, p < 0.001, η² = {eta_squared:.3f}")
print(f"• All pairwise comparisons significant (Bonferroni-corrected)")
print(f"• Performance hierarchy:")
for i, rc in enumerate(rider_order):
    mean_pts = rider_stats.loc[rc, 'Mean']
    print(f"    {i+1}. {rc}: {mean_pts:.2f} points (mean)")

print("\n--- RQ2: Rider-Stage Interaction ---")
print("• Specialization patterns confirmed:")
print(f"    - Sprinters: Flat ({interaction_means.loc['flat', 'Sprinter']:.2f}) >> "
      f"Mountain ({interaction_means.loc['mount', 'Sprinter']:.2f}) [ratio 19:1]")
print(f"    - Climbers: Mountain ({interaction_means.loc['mount', 'Climber']:.2f}) >> "
      f"Flat ({interaction_means.loc['flat', 'Climber']:.2f}) [ratio 7:1]")
print(f"    - All Rounders: Consistent, peak on mountains ({interaction_means.loc['mount', 'All Rounder']:.2f})")
print(f"    - Unclassed: Uniformly low performance")

print("\n" + "="*80)

## 8. Export Results

In [ ]:
# Export tables to CSV
rider_stats.to_csv('table_rider_stats.csv')
interaction_display.to_csv('table_interaction_means.csv')
pd.DataFrame(posthoc_results).to_csv('table_posthoc_results.csv', index=False)

print("Results exported:")
print("  ✓ table_rider_stats.csv")
print("  ✓ table_interaction_means.csv")
print("  ✓ table_posthoc_results.csv")
print("\nFigures saved:")
print("  ✓ fig1_pie_chart.pdf")
print("  ✓ fig2_violin_plot.pdf")
print("  ✓ fig3_heatmap.pdf")
print("  ✓ fig4_barchart.pdf")
print("  ✓ fig5_boxplot.pdf")

---
## References

1. Bland, J.M. & Altman, D.G. (1995). Multiple significance tests: The Bonferroni method. *BMJ*, 310, 170.
2. Cohen, J. (1988). *Statistical Power Analysis for the Behavioral Sciences*. Lawrence Erlbaum.
3. Kruskal, W.H. & Wallis, W.A. (1952). Use of ranks in one-criterion variance analysis. *J. Am. Stat. Assoc.*, 47, 583–621.
4. Mann, H.B. & Whitney, D.R. (1947). On a test of whether one of two random variables is stochastically larger. *Ann. Math. Stat.*, 18, 50–60.
5. Shapiro, S.S. & Wilk, M.B. (1965). An analysis of variance test for normality. *Biometrika*, 52, 591–611.
6. Tomczak, M. & Tomczak, E. (2014). The need to report effect size estimates revisited. *Trends in Sport Sci.*, 21, 19–25.